In [2]:
import os

# [Student: Mohamed] Create folder 'data' if it doesn't exist
folder_path = "data"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created.")
else:
    print(f"Folder '{folder_path}' already exists.")


Folder 'data' created.


In [ ]:
# [Student: Mohamed] Simulating 3000 transactions 
import pandas as pd
import random

item_pool = [
    "Milk", "Bread", "Eggs", "Butter", "Cheese", "Apples", "Bananas", "Oranges",
    "Chicken", "Beef", "Pasta", "Rice", "Tomatoes", "Onions", "Potatoes",
    "Lettuce", "Cucumber", "Carrots", "Yogurt", "Juice", "Cereal", "Coffee",
    "Tea", "Sugar", "Salt", "Cooking Oil", "Flour", "Toilet Paper", "Soap", "Shampoo"
]

transactions = []
for _ in range(3000):
    num_items = random.randint(2, 7)
    transaction = random.sample(item_pool, num_items)
    transactions.append(transaction)

# Save as CSV
df_transactions = pd.DataFrame({'Transaction': transactions})
df_transactions.to_csv('data/supermarket_transactions.csv', index=False)
df_transactions.head()  # Displaying the first few transactions

,Transaction
0,"[Cereal, Toilet Paper, Tea]"
1,"[Carrots, Tea, Bananas]"
2,"[Cooking Oil, Oranges, Eggs, Cheese, Tomatoes]"
3,"[Coffee, Salt, Bananas, Rice]"
4,"[Beef, Tea]"
